In [13]:
db_file = "/mnt/disk1/Github/total.db"

"/mnt/disk1/Github/total.db"

In [56]:
using SQLite;
using DataFrames;
using PyFormattedStrings;

In [16]:
db = SQLite.DB(db_file)

SQLite.DB("/mnt/disk1/Github/total.db")

In [17]:
SQLite.tables(db)

3-element Vector{SQLite.DBTable}:
 SQLite.DBTable("hyph", Tables.Schema:
 :first   Union{Missing, String}
 :second  Union{Missing, String})
 SQLite.DBTable("hyphendata", Tables.Schema:
 :first   Union{Missing, String}
 :second  Union{Missing, String}
 :word    Union{Missing, String}
 :freq    Union{Missing, Int64})
 SQLite.DBTable("words", Tables.Schema:
 :word  Union{Missing, String}
 :freq  Union{Missing, Int64})

In [37]:
function query(db, sql, params=())
        data = SQLite.DBInterface.execute(db, sql, params) |> DataFrame
    return data
end

query (generic function with 2 methods)

In [42]:
@time begin
    query(db, "select * from hyphendata where word = 'Nasjonalbiblioteket'")
end

  0.000347 seconds (273 allocations: 12.805 KiB)


Row,first,second,word,freq
,String,String,String,Int64
1,Nasjona,lbiblioteket,Nasjonalbiblioteket,2
2,Nasjonalbibliotek,et,Nasjonalbiblioteket,12
3,Nasjon,albiblioteket,Nasjonalbiblioteket,14
4,Nas,jonalbiblioteket,Nasjonalbiblioteket,39
5,Nasjonalbib,lioteket,Nasjonalbiblioteket,398
6,Nasjonalbibli,oteket,Nasjonalbiblioteket,419
7,Na,sjonalbiblioteket,Nasjonalbiblioteket,1108
8,Nasjonalbi,blioteket,Nasjonalbiblioteket,1301
9,Nasjonalbibliote,ket,Nasjonalbiblioteket,1357


In [55]:
query(db, "select rowid from words order by rowid desc limit 1")[!,"rowid"][1]

223039157

In [59]:
function bisectiterate(db, word)
    
    minrow = query(db, "select rowid from words order by rowid asc limit 1")[!,"rowid"][1]
    maxrow = query(db, "select rowid from words order by rowid desc limit 1")[!, "rowid"][1]
    
    return bisect(db, word, minrow, maxrow)
end

bisectiterate (generic function with 1 method)

In [134]:
function bisect(db, word, minrow, maxrow)

    found = false
    fromrow = minrow
    torow = maxrow
    res = []
    i = 0
    delta = 0
    limit = log2(maxrow + 1)
    
    while !found && (fromrow <= torow) && i < limit
        
        i += 1
        
        midrow = fromrow + div(torow - fromrow, 2)
        
        targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[!,"rowid"][1]
    
        if word == targeted[1]
            
            append!(res, targeted)
            
            found = true

            leftcandidate = midrow - 1
            lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[!,"rowid"][1]
            
            while lcandidate[1] == word and leftcandidate >= fromrow
                append!(res, lcandidate)
                leftcandidate -= 1
                lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[!,"rowid"][1]
            end
            
            rightcandidate = midrow + 1
            rcandidate = query(db, f"select word, freq, rowid from words where rowid={rightcandidate}")[!,"rowid"][1]
            
            while rcandidate[1] == word and rightcandidate <= torow
                append!(res, rcandidate)
                rightcandidate += 1
                rcandidate = query(db, f"select word, freq, rowid from words where rowid={rightcandidate}")[!,"rowid"][1]
            end
            
        elseif word > targeted[1]
            torow = midrow
        else
            fromrow = midrow 
        end
        
    end
    return res
end

LoadError: syntax: "while" at In[134]:28 expected "end", got "leftcandidate"

In [73]:
div(3,2)

1

In [4]:
function f(a, b)
    return a + b
end

f (generic function with 1 method)

In [9]:
@time begin
f(2,3)
end

  0.000001 seconds


5

In [116]:
#r = ["a","b"]
x = 3
y = 4
if x < 5 && y == "a"
    append!(r, "a")
    found = true
    
end

In [117]:
r

2-element Vector{String}:
 "a"
 "b"

In [118]:
f"here is {r[2]} and {x}"

"here is b and 3"

In [132]:
fromrow=1; torow=200030000

200030000

In [133]:
word = "sove"
midrow = 1 + div(torow - fromrow, 2)
targeted = query(db, f"select word, freq, rowid from words where rowid={midrow}")[!,"rowid"][1]
res = []
if word == targeted[1]

    append!(res, targeted)
else
    append!(res, [word])
    append!(res, ["hm"])
    leftcandidate = midrow - 1
    lcandidate = query(db, f"select word, freq, rowid from words where rowid={leftcandidate}")[!,"rowid"][1]
    println(lcandidate)
end
println(res)

100014999
Any["sove", "hm"]
